# Training & Validation notebook

This notebook walks through our training and validation scripts.

### Setting up the environment
Create a new virtual environment using `venv` or if already created, move on to the next part (skip this if running on Google colab).

In [ ]:
# !python3 -m venv venv
# !source venv/bin/activate

### Cloning the repo
Clone the repo and install the required packages defined in `requirements.txt`.

In [1]:
!git clone https://github.com/arnavrneo/torchFlow.git
%cd torchFlow
!pip install -r requirements.txt

Cloning into 'torchFlow'...
remote: Enumerating objects: 302, done.
remote: Counting objects: 100% (302/302), done.
remote: Compressing objects: 100% (252/252), done. (235/252)
remote: Total 302 (delta 95), reused 236 (delta 47), pack-reused 0
Receiving objects: 100% (302/302), 554.47 KiB | 2.27 MiB/s, done.
Resolving deltas: 100% (95/95), done.
/home/neo/Downloads/torchFlow



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


Run the following shell script to (skip this if the data is already arranged):
- download the `yolo` and `onnx` model checkpoints to the `models` directory;
- download the tiled datasets in their respective directories.

In [2]:
!./get-dataset.sh

Script Working
--2023-07-04 13:01:30--  https://www.dropbox.com/s/3k2pt9qh36vcvqb/dataset-original.zip
Resolving www.dropbox.com (www.dropbox.com)... 2620:100:6031:18::a27d:5112, 162.125.81.18
Connecting to www.dropbox.com (www.dropbox.com)|2620:100:6031:18::a27d:5112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/3k2pt9qh36vcvqb/dataset-original.zip [following]
--2023-07-04 13:01:31--  https://www.dropbox.com/s/raw/3k2pt9qh36vcvqb/dataset-original.zip
Reusing existing connection to [www.dropbox.com]:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6cac0505a782bdb65d99aac84e.dl.dropboxusercontent.com/cd/0/inline/B_NS1aHT6Cvd44uHWr-8aH_Wmxv-z8bB7QjMPpqpwMGmce5kmAElt1hnRxpuicWZ_llRJoKftyk4G8yZat7AI1KxBFVi4q9xDOW2eXSZSE5DpkQqfEiCM4vTLxJfurTdT_SNtzKIa6mqv9mEZpFZeqkN_QoDwDXtBWlXqlxe_nvcyA/file# [following]
--2023-07-04 13:01:31--  https://uc6cac0505a782bdb65d99aac84e.dl.dropboxusercontent.com/cd/0/inline/B_NS1aHT6Cvd44uHWr-8aH

HTTP request sent, awaiting response... 302 Found
Location: https://ucfa935f14a26603bd84051b6576.dl.dropboxusercontent.com/cd/0/inline/B_NJ8YoW24ozBb682iYide6_IiVdTW0FI5Xusa5H9_EeFw7RFQx598GCp5otzQbc4IK0jwytIMm6YvNxjRSyX6u18zUCrW17bpEOAqLLAdd1RgqgKMHowk-tvoiwj_UQO98mMMlle7UsACczHDd9nacRl7xI1UofSVCFhVaYVd8LeA/file# [following]
--2023-07-04 13:03:51--  https://ucfa935f14a26603bd84051b6576.dl.dropboxusercontent.com/cd/0/inline/B_NJ8YoW24ozBb682iYide6_IiVdTW0FI5Xusa5H9_EeFw7RFQx598GCp5otzQbc4IK0jwytIMm6YvNxjRSyX6u18zUCrW17bpEOAqLLAdd1RgqgKMHowk-tvoiwj_UQO98mMMlle7UsACczHDd9nacRl7xI1UofSVCFhVaYVd8LeA/file
Resolving ucfa935f14a26603bd84051b6576.dl.dropboxusercontent.com (ucfa935f14a26603bd84051b6576.dl.dropboxusercontent.com)... 2620:100:6031:15::a27d:510f, 162.125.81.15
Connecting to ucfa935f14a26603bd84051b6576.dl.dropboxusercontent.com (ucfa935f14a26603bd84051b6576.dl.dropboxusercontent.com)|2620:100:6031:15::a27d:510f|:443... connected.
HTTP request sent, awaiting response... 302 Found
L

  inflating: dataset-256/images/test/5e721561-DJI_0356_8_5.jpg  
  inflating: dataset-256/images/test/5e721561-DJI_0356_9_5.jpg  
  inflating: dataset-256/images/test/5e721561-DJI_0356_9_8.jpg  
  inflating: dataset-256/images/test/6b0c5198-DJI_0537_3_10.jpg  
  inflating: dataset-256/images/test/6b0c5198-DJI_0537_3_6.jpg  
  inflating: dataset-256/images/test/6b0c5198-DJI_0537_5_2.jpg  
  inflating: dataset-256/images/test/6b0c5198-DJI_0537_5_4.jpg  
  inflating: dataset-256/images/test/6b0c5198-DJI_0537_7_11.jpg  
  inflating: dataset-256/images/test/6f8c1717-DJI_0317_5_14.jpg  
  inflating: dataset-256/images/test/6f8c1717-DJI_0317_8_0.jpg  
  inflating: dataset-256/images/test/6f8c1717-DJI_0317_8_10.jpg  
  inflating: dataset-256/images/test/6f8c1717-DJI_0317_8_11.jpg  
  inflating: dataset-256/images/test/6f8c1717-DJI_0317_8_9.jpg  
  inflating: dataset-256/images/test/6f8c1717-DJI_0317_9_10.jpg  
  inflating: dataset-256/images/test/6f8c1717-DJI_0317_9_11.jpg  
  inflating: datas

  inflating: dataset-256/images/train/3004be3b-DJI_0250_8_5.jpg  
  inflating: dataset-256/images/train/3004be3b-DJI_0250_8_7.jpg  
  inflating: dataset-256/images/train/3004be3b-DJI_0250_9_0.jpg  
  inflating: dataset-256/images/train/3004be3b-DJI_0250_9_11.jpg  
  inflating: dataset-256/images/train/30a2b8ba-DJI_0160_0_1.jpg  
  inflating: dataset-256/images/train/30a2b8ba-DJI_0160_0_2.jpg  
  inflating: dataset-256/images/train/30a2b8ba-DJI_0160_1_1.jpg  
  inflating: dataset-256/images/train/30a2b8ba-DJI_0160_3_0.jpg  
  inflating: dataset-256/images/train/30a2b8ba-DJI_0160_3_1.jpg  
  inflating: dataset-256/images/train/30a2b8ba-DJI_0160_3_2.jpg  
  inflating: dataset-256/images/train/30a2b8ba-DJI_0160_4_0.jpg  
  inflating: dataset-256/images/train/30a2b8ba-DJI_0160_4_1.jpg  
  inflating: dataset-256/images/train/30a2b8ba-DJI_0160_4_13.jpg  
  inflating: dataset-256/images/train/30a2b8ba-DJI_0160_4_14.jpg  
  inflating: dataset-256/images/train/30a2b8ba-DJI_0160_5_13.jpg  
  infl

  inflating: dataset-256/images/train/ac9a9101-DJI_0302_7_8.jpg  
  inflating: dataset-256/images/train/ac9a9101-DJI_0302_7_9.jpg  
  inflating: dataset-256/images/train/ac9a9101-DJI_0302_8_12.jpg  
  inflating: dataset-256/images/train/ac9a9101-DJI_0302_8_14.jpg  
  inflating: dataset-256/images/train/b281a8cc-DJI_0175_0_10.jpg  
  inflating: dataset-256/images/train/b281a8cc-DJI_0175_0_9.jpg  
  inflating: dataset-256/images/train/b281a8cc-DJI_0175_3_13.jpg  
  inflating: dataset-256/images/train/b281a8cc-DJI_0175_4_11.jpg  
  inflating: dataset-256/images/train/b281a8cc-DJI_0175_4_12.jpg  
  inflating: dataset-256/images/train/b281a8cc-DJI_0175_5_11.jpg  
  inflating: dataset-256/images/train/b281a8cc-DJI_0175_5_12.jpg  
  inflating: dataset-256/images/train/b281a8cc-DJI_0175_6_10.jpg  
  inflating: dataset-256/images/train/b281a8cc-DJI_0175_7_10.jpg  
  inflating: dataset-256/images/train/b281a8cc-DJI_0175_8_8.jpg  
  inflating: dataset-256/images/train/b281a8cc-DJI_0175_9_8.jpg  


...

  inflating: dataset-1280/images/test/97a0c903-DJI_0310_1_0.jpg  
  inflating: dataset-1280/images/test/97a0c903-DJI_0310_1_1.jpg  
  inflating: dataset-1280/images/test/97a0c903-DJI_0310_1_2.jpg  
  inflating: dataset-1280/images/test/a68bb4a4-DJI_0449_0_0.jpg  
  inflating: dataset-1280/images/test/a68bb4a4-DJI_0449_0_2.jpg  
  inflating: dataset-1280/images/test/a68bb4a4-DJI_0449_1_2.jpg  
  inflating: dataset-1280/images/test/a6fedec6-DJI_0504_0_0.jpg  
  inflating: dataset-1280/images/test/a6fedec6-DJI_0504_0_1.jpg  
  inflating: dataset-1280/images/test/a6fedec6-DJI_0504_0_2.jpg  
  inflating: dataset-1280/images/test/a6fedec6-DJI_0504_1_0.jpg  
  inflating: dataset-1280/images/test/a6fedec6-DJI_0504_1_1.jpg  
  inflating: dataset-1280/images/test/a6fedec6-DJI_0504_1_2.jpg  
  inflating: dataset-1280/images/test/aabf4d74-DJI_0497_0_0.jpg  
  inflating: dataset-1280/images/test/aabf4d74-DJI_0497_0_1.jpg  
  inflating: dataset-1280/images/test/aabf4d74-DJI_0497_1_0.jpg  
  inflatin

  inflating: dataset-1280/images/train/5d430c63-DJI_0079_1_0.jpg  
  inflating: dataset-1280/images/train/5d430c63-DJI_0079_1_2.jpg  
  inflating: dataset-1280/images/train/5f1b5fe9-DJI_0176_0_0.jpg  
  inflating: dataset-1280/images/train/5f1b5fe9-DJI_0176_1_1.jpg  
  inflating: dataset-1280/images/train/5f1b5fe9-DJI_0176_1_2.jpg  
  inflating: dataset-1280/images/train/65334a32-DJI_0186_0_1.jpg  
  inflating: dataset-1280/images/train/65334a32-DJI_0186_0_2.jpg  
  inflating: dataset-1280/images/train/65334a32-DJI_0186_1_1.jpg  
  inflating: dataset-1280/images/train/65334a32-DJI_0186_1_2.jpg  
  inflating: dataset-1280/images/train/69a24132-DJI_0234_0_0.jpg  
  inflating: dataset-1280/images/train/69a24132-DJI_0234_0_1.jpg  
  inflating: dataset-1280/images/train/69a24132-DJI_0234_1_0.jpg  
  inflating: dataset-1280/images/train/69a24132-DJI_0234_1_1.jpg  
  inflating: dataset-1280/images/train/69a24132-DJI_0234_1_2.jpg  
  inflating: dataset-1280/images/train/6c846707-DJI_0044_1_2.j

  inflating: dataset-1280/images/train/f3604336-DJI_0270_0_1.jpg  
  inflating: dataset-1280/images/train/f3604336-DJI_0270_1_0.jpg  
  inflating: dataset-1280/images/train/f3604336-DJI_0270_1_1.jpg  
  inflating: dataset-1280/images/train/f3604336-DJI_0270_1_2.jpg  
  inflating: dataset-1280/images/train/f538ec0e-DJI_0165_0_0.jpg  
  inflating: dataset-1280/images/train/f538ec0e-DJI_0165_0_1.jpg  
  inflating: dataset-1280/images/train/f538ec0e-DJI_0165_0_2.jpg  
  inflating: dataset-1280/images/train/f538ec0e-DJI_0165_1_0.jpg  
  inflating: dataset-1280/images/train/f538ec0e-DJI_0165_1_1.jpg  
  inflating: dataset-1280/images/train/f538ec0e-DJI_0165_1_2.jpg  
  inflating: dataset-1280/images/train/f922e013-DJI_0145_0_0.jpg  
  inflating: dataset-1280/images/train/f922e013-DJI_0145_0_1.jpg  
  inflating: dataset-1280/images/train/f922e013-DJI_0145_1_2.jpg  
  inflating: dataset-1280/images/train/fb32cb77-DJI_0181_1_0.jpg  
  inflating: dataset-1280/images/train/fb32cb77-DJI_0181_1_1.j

  inflating: dataset-1280/labels/train/30a2b8ba-DJI_0160_1_1.txt  
  inflating: dataset-1280/labels/train/30a2b8ba-DJI_0160_1_2.txt  
  inflating: dataset-1280/labels/train/35a1a3cc-DJI_0021_0_1.txt  
  inflating: dataset-1280/labels/train/35a1a3cc-DJI_0021_1_1.txt  
  inflating: dataset-1280/labels/train/35a1a3cc-DJI_0021_1_2.txt  
  inflating: dataset-1280/labels/train/3af948c7-DJI_0136_0_1.txt  
  inflating: dataset-1280/labels/train/3af948c7-DJI_0136_1_0.txt  
  inflating: dataset-1280/labels/train/3af948c7-DJI_0136_1_1.txt  
  inflating: dataset-1280/labels/train/3fbaf47e-DJI_0183_0_0.txt  
  inflating: dataset-1280/labels/train/3fbaf47e-DJI_0183_0_1.txt  
  inflating: dataset-1280/labels/train/3fbaf47e-DJI_0183_1_0.txt  
  inflating: dataset-1280/labels/train/3fbaf47e-DJI_0183_1_1.txt  
  inflating: dataset-1280/labels/train/3fbaf47e-DJI_0183_1_2.txt  
  inflating: dataset-1280/labels/train/4d6bde3a-DJI_0184_0_1.txt  
  inflating: dataset-1280/labels/train/4d6bde3a-DJI_0184_1_0.t

Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 2620:100:6031:15::a27d:510f, 162.125.81.15
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|2620:100:6031:15::a27d:510f|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 178787201 (171M) [text/plain]
Saving to: ‘./models/8l-1280-3232.onnx’

8l-1280-3232.onnx   100%[===================>] 170.50M  12.5MB/s    in 17s     

2023-07-04 13:04:43 (9.91 MB/s) - ‘./models/8l-1280-3232.onnx’ saved [178787201/178787201]

--2023-07-04 13:04:43--  https://dl.dropboxusercontent.com/s/dv916v4t5hn6qpu/torchFlow-ckpt.pt
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 2620:100:6031:15::a27d:510f, 162.125.81.15
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|2620:100:6031:15::a27d:510f|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88572542 (84M) [application/octet-stream]
Saving to: ‘./models/torchFlow-ckpt.pt’

torchFlow-ckpt.pt 

In [3]:
!python config/config.py # for setting up the dataset directorys

### Training

For training, we have trained the model by tiling the dataset into the following sizes:
- 256 x 256
- 512 x 512
- 1280 x 1280

and recursively training the model on next tiled dataset and then finally training at 3200 x 2600 size on the original dataset.

The base model used: `yolov8l.pt`.

For replicating training process, keep changing:
- the model sizes acc. to the tile size (keep the params as it is) in the `train-config.yaml` file.
![epochs-sizes.png](assets/epochs-sizes.png)


- and the dataset directory path in the `dataset.yaml`, i.e.

![dataset-yaml.png](assets/dataset-yaml.png)

- and the model checkpoints.

In [ ]:
!python train.py -m yolov8l.pt

### Validation

Validation will be done automatically during the training step, but for manual validation, we provide the `val.py` script.
- We run the validation at `3232` size. The values can be changed in the `val-config.yaml`.

In [4]:
!python val.py -m /home/neo/Downloads/torchFlow/models/torchFlow-ckpt.pt

Running on: cuda
Ultralytics YOLOv8.0.121 🚀 Python-3.10.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1650 Ti, 3904MiB)
Model summary (fused): 268 layers, 43607379 parameters, 0 gradients
val: Scanning /home/neo/Downloads/torchFlow/dataset/labels/test... 30 images, 0 
val: New cache created: /home/neo/Downloads/torchFlow/dataset/labels/test.cache
                 Class     Images  Instances      Box(P          R      mAP50  m
Traceback (most recent call last):
  File "/home/neo/Downloads/torchFlow/val.py", line 36, in <module>
    val(
  File "/home/neo/Downloads/torchFlow/val.py", line 27, in val
    model.val(
  File "/home/neo/Downloads/ultralytics/venv/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/home/neo/Downloads/torchFlow/ultralytics/yolo/engine/model.py", line 304, in val
    validator(model=self.model)
  File "/home/neo/Downloads/ultralytics/venv/lib/python3.10/site-packages/torch/utils/_con